In [1]:
import xarray as xr 
import numpy as np
import dask
from dask.distributed import Client
import time

In [2]:
import multiprocessing
ncpu = multiprocessing.cpu_count()
processes = True
nworker = 16
threads = ncpu // nworker
print(
    f"Number of CPUs: {ncpu}, number of threads: {threads}, number of workers: {nworker}, processes: {processes}",
)

Number of CPUs: 256, number of threads: 16, number of workers: 16, processes: True


In [3]:
client = Client(
    processes=processes,
    threads_per_worker=threads,
    n_workers=nworker,
    memory_limit="24GB",
)
dask.config.config.get('distributed').get('dashboard').update({'link':'{JUPYTERHUB_SERVICE_PREFIX}/proxy/{port}/status'})

client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /user/b382641/levante-spawner-preset//proxy/8787/status,
Dashboard: /user/b382641/levante-spawner-preset//proxy/8787/status,Workers: 16
Total threads: 256,Total memory: 357.63 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:34045,Workers: 16
Dashboard: /user/b382641/levante-spawner-preset//proxy/8787/status,Total threads: 256
Started: Just now,Total memory: 357.63 GiB
Comm: tcp://127.0.0.1:40843,Total threads: 16
Dashboard: /user/b382641/levante-spawner-preset//proxy/43347/status,Memory: 22.35 GiB
Nanny: tcp://127.0.0.1:45389,


In [10]:
# paths = [
#     "/work/ik1017/CMIP6/data/CMIP6/ScenarioMIP/MPI-M/MPI-ESM1-2-LR/ssp370/r13i1p1f1/6hrLev/hus/gn/v20210901/hus_6hrLev_MPI-ESM1-2-LR_ssp370_r13i1p1f1_gn_201501010600-203501010000.nc",
#     "/work/ik1017/CMIP6/data/CMIP6/ScenarioMIP/MPI-M/MPI-ESM1-2-LR/ssp370/r13i1p1f1/6hrLev/ua/gn/v20210901/ua_6hrLev_MPI-ESM1-2-LR_ssp370_r13i1p1f1_gn_201501010600-203501010000.nc",
#     "/work/ik1017/CMIP6/data/CMIP6/ScenarioMIP/MPI-M/MPI-ESM1-2-LR/ssp370/r13i1p1f1/6hrLev/va/gn/v20210901/va_6hrLev_MPI-ESM1-2-LR_ssp370_r13i1p1f1_gn_201501010600-203501010000.nc" 
# ]

paths = ["/work/ik1017/CMIP6/data/CMIP6/ScenarioMIP/MPI-M/MPI-ESM1-2-LR/ssp585/r10i1p1f1/6hrLev/hus/gn/v20190710/hus_6hrLev_MPI-ESM1-2-LR_ssp585_r10i1p1f1_gn_201501010600-203501010000.nc", "/work/ik1017/CMIP6/data/CMIP6/ScenarioMIP/MPI-M/MPI-ESM1-2-LR/ssp585/r10i1p1f1/6hrLev/va/gn/v20190815/va_6hrLev_MPI-ESM1-2-LR_ssp585_r10i1p1f1_gn_201501010600-203501010000.nc", "/work/ik1017/CMIP6/data/CMIP6/ScenarioMIP/MPI-M/MPI-ESM1-2-LR/ssp585/r10i1p1f1/6hrLev/ua/gn/v20190815/ua_6hrLev_MPI-ESM1-2-LR_ssp585_r10i1p1f1_gn_201501010600-203501010000.nc"]

In [28]:
def integrate_trapz(array, x):
    return np.trapz(array, x, axis=-1) / 9.806

def integrate_over_data_array(y_values, x_values, integration_dim = 'lev'):
    return xr.apply_ufunc(
        integrate_trapz,
        y_values,
        x_values,
        input_core_dims=[[integration_dim], [integration_dim]],
        vectorize=True,
        dask='parallelized',
        output_dtypes=[float]  # Specify the output data type
     )

In [29]:
start_time = time.perf_counter()

ds = xr.open_mfdataset(paths, compat="override",  decode_cf=False, parallel=True, chunks=dict(time=256, lev= 47, lat= 96, lon= 192))
# ds = xr.open_mfdataset(paths, compat="override",  decode_cf=False, chunks=dict(time=128, lev= 47, lat= 96, lon= 192))

opened_datasets = time.perf_counter()
print(f"Time used for loading dataset: {opened_datasets - start_time} s")


Time used for loading dataset: 4.508274540014099 s


In [30]:


ds.coords["lon"] = (ds.coords["lon"] + 180) % 360 - 180
    
relevant_subset = ds.sortby(ds.lon).sel(lon=slice(-90, 40), lat=slice(20, 80))

relevant_subset["plevs"] = relevant_subset["ap"] + relevant_subset["b"] * relevant_subset["ps"]

eastward_product = relevant_subset["hus"] * relevant_subset["ua"]

northward_product = relevant_subset["hus"] * relevant_subset["va"]

eastward_integral = integrate_over_data_array(eastward_product, relevant_subset.plevs)

northward_integral = integrate_over_data_array(northward_product, relevant_subset.plevs)

ivt_norm = np.sqrt(eastward_integral**2 + northward_integral**2)
new_ds = xr.Dataset(coords={coord: relevant_subset[coord] for coord in relevant_subset.coords if coord != 'lev'})

new_ds.attrs = relevant_subset.attrs

new_ds["eastward_integral"] = eastward_integral
new_ds["northward_integral"] = northward_integral
new_ds["ivt"] = ivt_norm

/sw/spack-levante/mambaforge-22.9.0-2-Linux-x86_64-kptncg/lib/python3.10/site-packages/xarray/core/indexing.py:1440: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]
/sw/spack-levante/mambaforge-22.9.0-2-Linux-x86_64-kptncg/lib/python3.10/site-packages/xarray/core/indexing.py:1440: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    

In [19]:
relevant_subset.plevs

<xarray.DataArray 'plevs' (lev: 47, time: 29220, lat: 32, lon: 70)>
dask.array<add, shape=(47, 29220, 32, 70), dtype=float64, chunksize=(47, 256, 32, 70), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) float64 6.027e+04 6.027e+04 ... 6.757e+04 6.757e+04
  * lev      (lev) float64 0.9961 0.9826 0.959 ... 0.0001098 4.225e-05 9.816e-06
  * lat      (lat) float64 21.45 23.32 25.18 27.05 ... 73.68 75.54 77.41 79.27
  * lon      (lon) float64 -90.0 -88.12 -86.25 -84.38 ... 33.75 35.62 37.5 39.38

In [33]:
target_path = "/scratch/b/b382641/xarray_ssp585_r10i1p1f1_16_256_new_new.nc"

new_ds.to_netcdf(target_path)

very_end = time.perf_counter()

print(f"Time used for actual calculation: {very_end - opened_datasets} s")
print(f"Time used all in all: {(very_end - start_time)/60} min")

Time used for actual calculation: 5086.045588403009 s
Time used all in all: 84.84256438238371 min
